# Dataset Registry & Visualization using Amazong Glue Catalog & Amazon Athena

## 1. Import & Transform raw dataset

In [2]:
!aws s3 cp s3://$bucket/data/raw/BankChurners.csv ./Data Acquisition & Registry/data/BankChurners.csv


/bin/bash: line 1: Registry/data/BankChurners.csv: No such file or directory


In [2]:
import pandas as pd


local_data_path = './data/BankChurners.csv'
data = pd.read_csv(local_data_path)

data.shape


(10127, 23)

In [3]:
data.head()


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0,1
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0,1
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0,1
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0,1
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0,1


### Transofrm the data

In [4]:
data_transformed = data.rename(columns={
        'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1':'churn_mon1',
        'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2':'churn_mon2'
        })

data_transformed.dropna(inplace = True)
data_transformed.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 10127 non-null  int64  
 1   Attrition_Flag            10127 non-null  object 
 2   Customer_Age              10127 non-null  int64  
 3   Gender                    10127 non-null  object 
 4   Dependent_count           10127 non-null  int64  
 5   Education_Level           10127 non-null  object 
 6   Marital_Status            10127 non-null  object 
 7   Income_Category           10127 non-null  object 
 8   Card_Category             10127 non-null  object 
 9   Months_on_book            10127 non-null  int64  
 10  Total_Relationship_Count  10127 non-null  int64  
 11  Months_Inactive_12_mon    10127 non-null  int64  
 12  Contacts_Count_12_mon     10127 non-null  int64  
 13  Credit_Limit              10127 non-null  float64
 14  Total_

### Write data to local CSV file

In [5]:
import os


local_directory = './data/DatasetRegistration'
outname1 = 'BankChurners_month1_transformed.csv'
outname2 = 'BankChurners_month2_transformed.csv'

try:
    data_transformed.drop(columns = ['churn_mon2']).to_csv(os.path.join(local_directory, outname1), index=False)
    data_transformed.drop(columns = ['churn_mon1']).to_csv(os.path.join(local_directory, outname2), index=False)
except OSError:
    os.mkdir(local_directory)
    data_transformed.drop(columns = ['churn_mon2']).to_csv(os.path.join(local_directory, outname1), index=False)
    data_transformed.drop(columns = ['churn_mon1']).to_csv(os.path.join(local_directory, outname2), index=False)
    

## 2. Register S3 Dataset as a table to enable querying

In [6]:
import os
import sagemaker
import logging
import boto3
import time
import json
import botocore
from botocore.exceptions import ClientError


# ========================== low-level service client of the boto3 session ==========================
config = botocore.config.Config(user_agent_extra='bedissj-1699438736259')


sm = boto3.client(service_name='sagemaker', 
                  config=config)

sm_runtime = boto3.client('sagemaker-runtime',
                          config=config)

sess = sagemaker.Session(sagemaker_client=sm,
                         sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Copy file to S3 Bucket

In [28]:
!aws s3 cp ./data/DatasetRegistration/BankChurners_month1_transformed.csv s3://$bucket/data/transformed_querying/month1/BankChurners_mon1.csv


upload: data/DatasetRegistration/BankChurners_month1_transformed.csv to s3://sagemaker-eu-west-3-668303144976/data/transformed_querying/month1/BankChurners_mon1.csv


In [29]:
!aws s3 cp ./data/DatasetRegistration/BankChurners_month2_transformed.csv s3://$bucket/data/transformed_querying/month2/BankChurners_mon2.csv


upload: data/DatasetRegistration/BankChurners_month2_transformed.csv to s3://sagemaker-eu-west-3-668303144976/data/transformed_querying/month2/BankChurners_mon2.csv


In [8]:
from IPython.display import display, HTML

display(HTML('<b>Review <a target="top" href="https://s3.console.aws.amazon.com/s3/buckets/{}?region={}&prefix=data/transformed_querying/&showversions=false">Amazon S3 buckets</a></b>'.format(bucket, region)))

### Import AWS Data Wrangler & Create AWS Glue Data Catalog

In [10]:
!pip install --disable-pip-version-check -q awswrangler

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.


Create database if it does not already exist.

In [26]:
import awswrangler as wr


database_name = 'bank_churn_prediction'
wr.catalog.create_database(
        name=database_name,
        description='Database for banks customers churn prediction over two months. Each month is represented by a table.',
        exist_ok=True
)


Review list of available databases.

In [27]:
wr.catalog.databases()


,Database,Description
0,bank_churn_prediction,Database for banks customers churn prediction ...
1,default,
2,sagemaker_featurestore,


Create CSV tables and assign the to the destination database.

Map columns types to match AWS Glue Catalog.

In [60]:
# Create dataframe and retrieve columns types

data_mon1 = data_transformed.drop(columns = ['churn_mon2'])
data_types = pd.DataFrame(data_mon1.dtypes.to_dict(), 
                              index = [0]).replace({'int64': 'int',
                                                    'object':'string',
                                                    'float64': 'float'}).T.to_dict()[0]


Create CSV tables for month1 and month2 churn prediction.

In [61]:
# Create CSV table for churn month1

wr.catalog.create_csv_table(
        database=database_name,
        table='bank-churn-month1',
        path='s3://{}/data/transformed_querying/month1/'.format(bucket),
        columns_types=data_types,
        skip_header_line_count=1
)


In [63]:
# Create CSV table for churn month 2

data_mon2 = data_transformed.drop(columns = ['churn_mon1'])
wr.catalog.create_csv_table(
        database=database_name,
        table='bank-churn-month2',
        path='s3://{}/data/transformed_querying/month2/'.format(bucket),
        columns_types=data_types,
        skip_header_line_count=1
)


## Create Athena Bucket for querying

In [65]:
wr.athena.create_athena_bucket()

's3://aws-athena-query-results-668303144976-eu-west-3/'

In [73]:
database_name = 'bank_churn_prediction'
table_name = 'bank-churn-month1'

In [80]:
statement_table = '''
SELECT * FROM {}
'''.format(table_name)

print(statement_table)


SELECT * FROM bank-churn-month1



In [82]:
# wr.athena.read_sql_query(
#         sql=statement_table,
#         database=database_name
# )